In [4]:
from json2html import *
import json

In [3]:
# j = json.load(open('./vulnerabilities/2022/AVID-2022-V001.json'))
j = json.load(open('Untitled.txt'))

In [4]:
with open("sample.json", "w") as outfile:
    outfile.write(json.dumps(j))

In [16]:
json2html.convert(json = j, table_attributes="id=\"info-table\" class=\"table table-bordered table-hover\"")

'<table id="info-table" class="table table-bordered table-hover"><tr><th>vuln_id</th><td>AVID-2022-V001</td></tr><tr><th>metadata</th><td><table id="info-table" class="table table-bordered table-hover"><tr><th>class</th><td>LLM evaluation</td></tr><tr><th>taxonomy_version</th><td>0.1</td></tr></table></td></tr><tr><th>description</th><td><table id="info-table" class="table table-bordered table-hover"><tr><th>lang</th><td>eng</td></tr><tr><th>name</th><td>Gender Bias in Sentence Completion Tasks performed by bert-base-uncased</td></tr><tr><th>value</th><td>xyz xyz</td></tr></table></td></tr><tr><th>reports</th><td><table id="info-table" class="table table-bordered table-hover"><thead><tr><th>report_id</th><th>class</th><th>name</th></tr></thead><tbody><tr><td>AVID-2022-R0001</td><td>Measurement</td><td>Gender Bias in sentence completion Tasks performed by bert-base-uncased using the HONEST metric</td></tr><tr><td>AVID-2022-R0003</td><td>Measurement</td><td>Profession bias reinforcing ge

In [6]:
j = json.load(open('test.json'))

In [8]:
import jsonschema2md

parser = jsonschema2md.Parser(
    examples_as_yaml=False,
    show_examples="all",
)
md_lines = parser.parse_schema(j)
print(''.join(md_lines))

# Person

*JSON Schema for a person object.*

## Properties

- **`firstName`** *(string)*: The person's first name.
- **`lastName`** *(string)*: The person's last name.

